In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature = 0.1,
    model = "gpt-4o-mini-2024-07-18",
)

filename = 'document.txt'

loader = UnstructuredFileLoader(f'../{filename}')
cache_dir = LocalFileStore('../cache/')
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = '\n\n',
    chunk_size = 1000,
    chunk_overlap = 100,
)


docs = loader.load_and_split(text_splitter = splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()
memory = ConversationBufferMemory(
    llm = llm,
    # max_token_limit = 120,
    memory_key = 'history',
    return_messages = True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","""
            당신은 문서를 받고, 그 문서를 파악하여 대답해주는 AI에요.
            만약 해당하는 정보를 문서에서 찾지 못하는 경우엔 '모르겠습니다.' 라고 답변해주세요.
            무슨 일이 있어도 한국어로만 대답해주세요.
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)



In [40]:
invoke_chain("Aaronson 은 유죄인가요?")
invoke_chain('그가 테이블에 어떤 메시지를 썼나요?')
invoke_chain('Julia 는 누구인가요?')

문서에서는 Aaronson이 유죄라고 언급되고 있습니다. 그가 범죄를 저질렀다고 주장되며, 그에 대한 사진이 존재하지 않았고, 그는 그것을 발명했다고 말하고 있습니다.
그는 테이블에 "2+2=5"라는 메시지를 썼습니다.
Julia는 문서에서 Winston의 사랑하는 사람으로 언급됩니다. 그녀는 Winston과 함께 있었던 과거의 연인으로, 그들의 관계는 당국에 의해 위협받고 있습니다.
